# Métrica 3_3_01 — Congruencia en fecha de adquisición de bienes inmuebles
| Escenario                                                            | Condición                             | Resultado        |
| :------------------------------------------------------------------- | :------------------------------------ | :--------------- |
| No hay bienes inmuebles                                              | —                                     | ⚪ **SIN_DATO**   |
| Fecha de adquisición posterior a la fecha de declaración             | `fechaAdquisicion > fechaDeclaracion` | 🔴 **NO_CUMPLE** |
| Fecha de adquisición anterior al ingreso al servicio público         | `fechaAdquisicion < fechaIngresoSP`   | 🔴 **NO_CUMPLE** |
| Fechas dentro del rango lógico (ingreso ≤ adquisición ≤ declaración) | —                                     | 🟢               |


In [ ]:
import traceback
from datetime import datetime
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "3_3_01_CONGRUENCIA_FECHA_BIENES_INMUEBLES"

DICT_INFO = {
    "bienes_path": "declaracion.situacionPatrimonial.bienesInmuebles.bienInmueble",
    "fecha_ingreso_path": "declaracion.situacionPatrimonial.datosEmpleoCargoComision.fechaIngreso",
    "fecha_declaracion_path": "declaracion.fechaRecepcion",
    "obligatorio": False
}

# --- Funciones auxiliares ---

def extraer_lista(doc, path):
    """Extrae una lista desde un path anidado."""
    try:
        partes = path.split(".")
        actuales = [doc]
        for p in partes:
            siguientes = []
            for actual in actuales:
                if isinstance(actual, dict):
                    valor = actual.get(p, None)
                    if valor is None:
                        continue
                    if isinstance(valor, list):
                        siguientes.extend(valor)
                    else:
                        siguientes.append(valor)
                elif isinstance(actual, list):
                    for sub in actual:
                        if isinstance(sub, dict):
                            valor = sub.get(p, None)
                            if valor is None:
                                continue
                            if isinstance(valor, list):
                                siguientes.extend(valor)
                            else:
                                siguientes.append(valor)
            actuales = siguientes
        return [a for a in actuales if isinstance(a, dict)]
    except Exception:
        return []


def extraer_valor(doc, path):
    """Extrae un valor simple (puede ser fecha o texto)."""
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p, None)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p, None)
            else:
                return None
        return actual
    except Exception:
        return None


def parsear_fecha(fecha_str):
    """Convierte string a objeto datetime."""
    if not fecha_str:
        return None
    try:
        return datetime.fromisoformat(fecha_str[:10])
    except Exception:
        try:
            return datetime.strptime(fecha_str, "%Y-%m-%d")
        except Exception:
            return None


# --- Evaluación principal ---

def evaluar_metrica(doc):
    bienes = extraer_lista(doc, DICT_INFO["bienes_path"])
    fecha_ingreso = parsear_fecha(extraer_valor(doc, DICT_INFO["fecha_ingreso_path"]))
    fecha_declaracion = parsear_fecha(extraer_valor(doc, DICT_INFO["fecha_declaracion_path"]))

    if not bienes:
        return "SIN_DATO"

    if not fecha_ingreso or not fecha_declaracion:
        return "SIN_DATO"

    for bien in bienes:
        fecha_adq = parsear_fecha(bien.get("fechaAdquisicion"))
        if not fecha_adq:
            continue

        # Adquisición antes del ingreso al SP
        if fecha_ingreso and fecha_adq < fecha_ingreso:
            return "NO_CUMPLE"

        # Adquisición después de la declaración
        if fecha_declaracion and fecha_adq > fecha_declaracion:
            return "NO_CUMPLE"

    return "CUMPLE"


# --- Procesamiento MongoDB ---

def procesar_metrica_3_3_01():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0}
    operaciones = []
    total = 0

    try:
        print(f"Procesando métrica {METRIC_ID}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            DICT_INFO["bienes_path"]: 1,
            DICT_INFO["fecha_ingreso_path"]: 1,
            DICT_INFO["fecha_declaracion_path"]: 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1
            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"Error en doc {doc.get('_id')}: {e}")

            resultados[resultado] = resultados.get(resultado, 0) + 1
            operaciones.append(UpdateOne(
                {"_id": doc["_id"]},
                {"$set": {METRIC_ID: resultado}},
                upsert=True
            ))

            if len(operaciones) >= 1000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f"  > Procesados {total} documentos...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Total procesados: {total}")
        for k, v in resultados.items():
            print(f"  > {k}: {v}")

    except Exception:
        traceback.print_exc()
    finally:
        try:
            client.close()
        except:
            pass
        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_3_3_01()
